In [1]:
import numpy as np

In [2]:
from cana.datasets.bio import TEMPY
from cana.datasets.bio import (
    THALIANA,
    DROSOPHILA,
    BUDDING_YEAST,
    MARQUESPITA,
    LEUKEMIA,
    BREAST_CANCER,
)
from cana.boolean_node import BooleanNode
from cana.drawing.schema_vis import plot_schemata
from cana.drawing.plot_look_up_table import plot_look_up_table
from cana.utils import fill_out_lut

SyntaxError: invalid syntax. Perhaps you forgot a comma? (schema_vis.py, line 93)

In [ ]:
# modified the BooleanNetwork.from_string_cnet function that is called in the BooleanNetwork.from_file function to check for duplicate values in the inputs and outputs of the nodes.
# this will print a warning message if there are duplicate values in the inputs and outputs of the nodes.
# This also generates a lookup table data for nodes with k>1

# plot_schemata(tempy.nodes[4])
# plot_look_up_table(tempy.nodes[4])

In [ ]:
thaliana = THALIANA()
drosophila = DROSOPHILA()
budding_yeast = BUDDING_YEAST()
marquespita = MARQUESPITA()
leukemia = LEUKEMIA()
breast_cancer = BREAST_CANCER()
tempy = TEMPY()

In [ ]:
# print(thaliana.nodes[0].outputs)
node = tempy.nodes[4]
plot_look_up_table(node)
plot_schemata(node)
print(node.schemata_look_up_table(type="pi"))

generating permutations of output lists for incomplete data

In [ ]:
# need to modify the output list from data function to include the this following function
def from_output_list_with_missing_data(outputs=list(), *args, **kwargs):
    """
    Instanciate a Boolean Node from a output transition list.

    For missing data labeled as '#', '-', None, or 'x': In this case, we replace the missing data with a placeholder value, such as '-'. This allows us to maintain the structure of the outputs list while indicating that the data is missing.

    Complete line missing: If a complete line is missing from the outputs list, we can generate the missing rows as incomplete data. This can be done by extending the outputs list with the placeholder value '-' until it reaches the expected length of 2^k, where k is the number of inputs.

    Args:
        outputs (list) : The transition outputs of the node.

    Returns:
        (BooleanNode) : the instanciated object.

    Example:
        >>> BooleanNode.from_output_list_with_missing_data(outputs=[0,0,0,'-',1], name="AND")
    """
    id = kwargs.pop("id") if "id" in kwargs else 0
    name = kwargs.pop("name") if "name" in kwargs else "x"
    k = int(np.ceil(np.log2(len(outputs))))
    inputs = kwargs.pop("inputs") if "inputs" in kwargs else [(x + 1) for x in range(k)]
    state = kwargs.pop("state") if "state" in kwargs else False

    # Replace 'None', '-', '#', or 'x' with '-'.
    for i in range(len(outputs)):
        if outputs[i] in [None, "-", "#", "x"]:
            outputs[i] = "-"  # Placeholder value for missing data
            print(
                "Some of the lines contain data in the form of 'x', '#', None or '-'. These have been replaced with the placeholder value '-'. for internal consistency."
            )

    # Generate extra lines in the table to account for missing lines
    if len(outputs) < 2**k:
        print(
            f"Some of the lines in the data are missing and have been replaced with the placeholder value '-' for upto 2^k lines. i.e. the total lines inputted are {len(outputs)}, then the function will generate the missing rows for upto 2^{k} = {2**k} lines."
        )
        outputs.extend(["-"] * (2**k - len(outputs)))

    return BooleanNode(
        id=id,
        name=name,
        k=k,
        inputs=inputs,
        state=state,
        outputs=outputs,
        *args,
        **kwargs,
    )


# def generate_output_list_permutations(incomplete_boolean_node):
#     # generating all possible output_list permutations for the incomplete boolean node.

#     outputs = incomplete_boolean_node.outputs
#     print(outputs)
#     missing_data_indices = [i for i, x in enumerate(outputs) if x == '-']
#     # output_list_permutations = [outputs.copy() for _ in range(2 ** len(missing_data_indices))]

#     #generating binarytable for size: no. of missing values
#     table=[]
#     output_list_permutations=[]
#     for i in range(2 ** len(missing_data_indices)):
#             row = [int(x) for x in bin(i)[2:].zfill(len(missing_data_indices))]
#             table.append(row)
#             output_list_permutations.append(outputs.copy())
#             for j in range(len(missing_data_indices)):
#                 output_list_permutations[i][missing_data_indices[j]] = table[i][j]
#     return output_list_permutations

In [ ]:
incomplete_output = [0, 0, None, 1, 1, 0]
# incomplete_output = tempy.nodes[5].outputs.copy()
incomplete_output


# creating a boolean node for incomplete data
incomplete_boolean_node = from_output_list_with_missing_data(incomplete_output)
print(incomplete_boolean_node)
# output_list_permutations = generate_output_list_permutations(incomplete_boolean_node)
# print(f'Following are the pemutations of the incomplete output list of length:{len(output_list_permutations)}')
# output_list_permutations

In [ ]:
incomplete_boolean_node.look_up_table()

Added missing data code to the from_output_list funciton in boolean_node.py. 

In [ ]:
# # testing modified from_output_list function
# incomplete_output = [0,0, None, 1,1,0]
# incomplete_boolean_node = BooleanNode.from_output_list(incomplete_output)
# print(incomplete_boolean_node.outputs)
# incomplete_boolean_node.look_up_table()
AND = BooleanNode.from_output_list([0, 1, 0, 1])
print(AND.schemata_look_up_table(type="pi"))
plot_look_up_table(AND)
print(AND.look_up_table())

In [ ]:
# # partial_lut = [('1--','1'),('101','0'),('011','0'),('01-','1')]
# # partial_lut = [('0--0','0'),('1--1','0'),('0111','1'),('0011','1')]
partial_lut = [("00--", "0"), ("1--1", "1"), ("11--", "0")]
generated_lut = fill_out_lut(partial_lut)
generated_lut

In [ ]:
# write a function that converts  datya in the example form 00-- 0\n1--1 1\n11-- 0 to this example form: ('00--','0'),('1--1','1'),('11--','0')
def from_example_form_to_list(example_form):
    example_form = example_form.split("\n")
    example_form = [tuple(x.split(" ")) for x in example_form]
    return example_form


print(from_example_form_to_list("###1 0\n#1## 0\n1### 0\n##1# 0\n0000 1"))


# write a function that converts data in the example form ('00--','0'),('1--1','1'),('11--','0') to this example form: 00-- 0\n1--1 1\n11-- 0
def from_list_to_example_form(example_form):
    example_form = [x[0] + " " + x[1] for x in example_form]
    example_form = "\n".join(example_form)
    return example_form


# print(from_list_to_example_form(partial_lut))